In [1]:
import os
from glob import glob
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets 
import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:500]
test_paths = glob('../dataset/cifar/test/*.png')[:500]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
# train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(load_image_label)
# train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
# test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(load_image_label)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

### Training

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
15/15 [==============================] - 5s 233ms/step - loss: 2.3876 - accuracy: 0.0915 - val_loss: 2.2997 - val_accuracy: 0.1646
Epoch 2/20
15/15 [==============================] - 2s 122ms/step - loss: 2.2894 - accuracy: 0.1261 - val_loss: 2.2949 - val_accuracy: 0.1167
Epoch 3/20
15/15 [==============================] - 2s 130ms/step - loss: 2.2890 - accuracy: 0.1232 - val_loss: 2.2781 - val_accuracy: 0.1375
Epoch 4/20
15/15 [==============================] - 2s 135ms/step - loss: 2.2541 - accuracy: 0.1478 - val_loss: 2.2179 - val_accuracy: 0.2083
Epoch 5/20
15/15 [==============================] - 2s 125ms/step - loss: 2.2074 - accuracy: 0.1803 - val_loss: 2.1337 - val_accuracy: 0.2042
Epoch 6/20
15/15 [==============================] - 2s 127ms/step - loss: 2.1743 - accuracy: 0.1492 - val_loss: 2.1388 - val_accuracy: 0.1542
Epoch 7/20
15/15 [==============================] - 2s 133ms/step - loss: 2.1713 - accuracy: 0.1743 - val_loss: 2.1505 - val_accuracy: 0.2146
Epoch 

### Predict

#### 1) 이미지를 직접 load해서 넣는 방법

In [15]:
path = test_paths[0]
path

'../dataset/cifar/test\\0_cat.png'

In [16]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype=tf.float32)
image.shape

TensorShape([32, 32, 3])

In [17]:
image = image[tf.newaxis, ...]
image.shape

TensorShape([1, 32, 32, 3])

In [18]:
pred = model.predict(image)
pred.shape

(1, 10)

In [20]:
np.argmax(pred)

3

### Generator에서 데이터를 가져오는 방법

In [21]:
test_image, test_label = next(iter(test_dataset))
test_image.shape

TensorShape([32, 32, 32, 3])

In [22]:
pred = model.predict(test_image)
pred.shape

(32, 10)

In [25]:
np.argmax(pred, axis=1)

array([3, 2, 0, 6, 2, 0, 9, 6, 8, 0, 6, 3, 0, 0, 4, 8, 4, 2, 1, 6, 0, 8,
       3, 9, 2, 0, 8, 8, 2, 4, 2, 8], dtype=int64)

### Generator에 넣는 방법

In [26]:
pred = model.predict_generator(test_dataset.take(1))
pred.shape

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


(32, 10)

In [27]:
np.argmax(pred, axis=1)

array([3, 2, 0, 6, 2, 0, 9, 6, 8, 0, 6, 3, 0, 0, 4, 8, 4, 2, 1, 6, 0, 8,
       3, 9, 2, 0, 8, 8, 2, 4, 2, 8], dtype=int64)

In [29]:
image, label = next(iter(test_dataset))
model.evaluate(image, label)

1/1 [==============================] - 0s 126ms/step - loss: 1.9995 - accuracy: 0.2812


[1.9994655847549438, 0.28125]